In [1]:
import torch
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('Bank_Personal_Loan_Modelling-Copy1.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [6]:
df.head()


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [7]:
df.drop("ZIP Code",axis=1,inplace=True)
df.drop("ID",axis=1,inplace=True)

In [8]:
df['target']=df['Personal Loan']


In [9]:
df.drop('Personal Loan',axis=1,inplace=True)

In [10]:
df["Experience"] = abs(df["Experience"])

In [11]:
df

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,target
0,25,1,49,4,1.6,1,0,1,0,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,3,0,0,0,1,0,0
4996,30,4,15,4,0.4,1,85,0,0,1,0,0
4997,63,39,24,2,0.3,3,0,0,0,0,0,0
4998,65,40,49,3,0.5,2,0,0,0,1,0,0


In [12]:
x = df.iloc[:,:-1].values
y= df.iloc[:,-1].values

In [13]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.3,random_state =0)

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [15]:
import torch
from torch.utils.data import DataLoader, TensorDataset
BATCH_SIZE = 2
train_x = torch.from_numpy(x_train).to(torch.float32)
train_y = torch.from_numpy(y_train).to(torch.float32)


In [16]:
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

In [17]:
test_x = torch.from_numpy(x_test).to(torch.float32)
test_y = torch.from_numpy(y_test).to(torch.float32)
test = TensorDataset(test_x,test_y)
test = DataLoader(test,batch_size=1)

In [18]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        
        self.layer1 = torch.nn.Linear(11,16)
        self.layer2 = torch.nn.Linear(16,1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

In [25]:
model = Model()

In [27]:
torch.manual_seed(6699)
torch.set_grad_enabled(False)

dummy = Model()
dummy_param = np.concatenate([i.numpy().flatten() for i in dummy.parameters()])
shape = [i.numpy().shape for i in model.parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]
dim = len(dummy_param)

print("Dim : ", dim)
print("Layers Shape : ", shape)
print("Layers Size : ", size)

dummy = None
dummy_param = None

Dim :  209
Layers Shape :  [(16, 11), (16,), (1, 16), (1,)]
Layers Size :  [176, 16, 16, 1]


In [28]:
def model_to_vector(model):
    vector = np.concatenate([i.numpy().flatten() for i in model.parameters()])
    return vector
    
def vector_to_model(vector):
    param = list()
    cum_sum = 0
    for i in range(len(size)):
        array = vector[cum_sum : cum_sum + size[i]]
        array = array.reshape(shape[i])
        cum_sum += size[i]
        param.append(array)
    param = np.array(param, dtype="object")
    
    dummy_model = Model()
    for idx,wei in enumerate(dummy_model.parameters()):
        wei.data = (torch.tensor(param[idx])).to(torch.float32)
        
    return dummy_model    

def calc_accuracy(model):
    y_pred = model(train_x)
    y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
    acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
    
    return acc

In [32]:
ants = 10
loops = 100
evaporation_rate = 0.2
influence_factor = 0.4
pheromones = np.ones(dim)
max_accuracy = 0
fittest_vector = None

for loop in range(loops):
    # Generate Solution
    paths = np.array([Model() for i in range(ants)])
    accuracy = []
    
    for ant in range(ants):
        # Flatten the weights and biases
        vector = model_to_vector(paths[ant])
        
        # Multiply with pheromones 
        vector = vector * pheromones
        
        # Calculate Accuracy and Append to the list
        model = vector_to_model(vector)
        acc = calc_accuracy(model)
        accuracy.append(acc)
        
        # Update the updated path
        paths[ant] = model
        
        # Reset
        model = None
        acc = None
paths = paths[np.argsort(accuracy)]
    
if accuracy[np.argmax(accuracy)] > max_accuracy:
    max_accuracy = accuracy[np.argmax(accuracy)]
    fittest_vector = model_to_vector(paths[-1])
    
   
delta = 0
for ant in range(ants):
        
    vector = model_to_vector(paths[ant])
        
       
    delta += (vector - fittest_vector)*influence_factor
        
    pheromones = (1-pheromones)*evaporation_rate + delta
    
if loop%10 == 0:
    print("Iters {} :".format(loop), calc_accuracy(paths[-1]))

In [33]:
print("Maximum Accuracy : ", max_accuracy)
best_model = vector_to_model(fittest_vector)

Maximum Accuracy :  0.8411428571428572


In [34]:
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,test_y))


              precision    recall  f1-score   support

           0       0.92      0.92      0.92      1368
           1       0.16      0.15      0.15       132

    accuracy                           0.85      1500
   macro avg       0.54      0.54      0.54      1500
weighted avg       0.85      0.85      0.85      1500



In [20]:
'''from pyswarms.single import GlobalBestPSO


torch.manual_seed(420)
torch.set_grad_enabled(False)

model = Model()
params = [i.numpy().flatten() for i in model.parameters()]
shapes = [i.numpy().shape for i in model.parameters()]
sizes = [i[0] * i[1] if len(i) == 2 else i[0] for i in shapes]

print(f"Dim: {sum(len(param) for param in params)}")
print(f"Layers Shape: {shapes}")
print(f"Layers Size: {sizes}")'''

Dim: 209
Layers Shape: [(16, 11), (16,), (1, 16), (1,)]
Layers Size: [176, 16, 16, 1]


In [21]:
'''def objective_function(particle, shape, size):
    accuracy = []
    output = []
    for par in particle:
        cum_sum = 0
        param = [par[cum_sum:cum_sum+size[i]].reshape(shape[i]) for i in range(len(size))]
        cum_sum += size[i]
        output.append(param)
    with torch.no_grad():
        for param in output:
            model = Model()
            for i, wei in enumerate(model.parameters()):
                wei.data = torch.tensor(param[i]).detach()
            y_pred = model(train_x)
            y_pred = torch.where(y_pred >= 0.5, 1, 0).flatten()
            acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
            accuracy.append(1 - acc)
    return accuracy
'''

In [ ]:
'''options = {'c1': 0.6, 'c2': 0.3, 'w': 0.1}
bounds = (-1.0, 1.0)
n_particles = 500
size = [param[i].numel() for i in range(len(param))]
shape = [param[i].shape for i in range(len(param))]
dimensions = sum(size)

pso = GlobalBestPSO(n_particles=n_particles, dimensions=dimensions, options=options, bounds=bounds)
best_cost, (best_param,) = pso.optimize(lambda x: objective_function(x, shape=shape, size=size), iters=50)
accuracy = 1 - best_cost
print(f"Accuracy: {accuracy:.4f}")
'''

In [23]:
'''best_param, = best_parameters
result = []
cum_sum = 0
for i in range(len(size)):
    array = best_param[cum_sum : cum_sum + size[i]]
    array = array.reshape(shape[i])
    cum_sum += size[i]
    result.append(array)
result = np.array(result, dtype="object")

best_model = Model()
with torch.no_grad():
    for idx, wei in enumerate(best_model.parameters()):
        wei.data = torch.tensor(result[idx]).float()

# Calculate Accuracy
y_pred = best_model(train_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
accuracy = (y_pred == train_y).sum().float().item() / len(data.dataset)
print(f"Accuracy: {accuracy:.4f}")
''''''

NameError: name 'best_parameters' is not defined

In [24]:
from sklearn.metrics import classification_report
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,test_y))

NameError: name 'best_model' is not defined